In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import json
import os
import shutil
import time

from pathlib import Path
from src.serving import export
from src.configs import EnsembleConfig

In [5]:
PROJECT_ID = 'jk-mlops-dev' # Change to your project.
REGION = 'us-central1'  # Change to your region.
STAGING_BUCKET = 'jk-merlin-dev' # Change to your bucket.
MODEL_REPOSITORY_BUCKET = 'jk-vertex-staging'
LOCAL_WORKSPACE = '/home/jupyter/staging'

MODEL_NAME = 'deepfm'
MODEL_VERSION = 'v01'
MODEL_DISPLAY_NAME = f'hugectr-{MODEL_NAME}-{MODEL_VERSION}'
MODEL_DESCRIPTION = 'HugeCTR DeepFM model'
ENDPOINT_DISPLAY_NAME = f'hugectr-{MODEL_NAME}-{MODEL_VERSION}'

EXPORTED_MODELS_DIR = f'gs://{MODEL_REPOSITORY_BUCKET}/hugectr_models'

IMAGE_NAME = 'triton-deploy-hugectr'
IMAGE_URI = f"gcr.io/{PROJECT_ID}/{IMAGE_NAME}"
DOCKERFILE = 'src/Dockerfile.triton'

WORKFLOW_MODEL_DIR = "gs://criteo-datasets/criteo_processed_parquet/workflow" # Change to GCS path of the nvt workflow.
HUGECTR_MODEL_DIR = "gs://merlin-models/hugectr_deepfm_21.09" # Change to GCS path of the hugectr trained model.

In [6]:
if os.path.isdir(LOCAL_WORKSPACE):
    shutil.rmtree(LOCAL_WORKSPACE)
os.makedirs(LOCAL_WORKSPACE)

!gsutil -m cp -r {WORKFLOW_MODEL_DIR} {LOCAL_WORKSPACE}
!gsutil -m cp -r {HUGECTR_MODEL_DIR} {LOCAL_WORKSPACE}

Copying gs://criteo-datasets/criteo_processed_parquet/workflow/categories/unique.C1.parquet...
Copying gs://criteo-datasets/criteo_processed_parquet/workflow/categories/unique.C10.parquet...
Copying gs://criteo-datasets/criteo_processed_parquet/workflow/categories/unique.C11.parquet...
Copying gs://criteo-datasets/criteo_processed_parquet/workflow/categories/unique.C12.parquet...
Copying gs://criteo-datasets/criteo_processed_parquet/workflow/categories/unique.C13.parquet...
Copying gs://criteo-datasets/criteo_processed_parquet/workflow/categories/unique.C15.parquet...
Copying gs://criteo-datasets/criteo_processed_parquet/workflow/categories/unique.C14.parquet...
Copying gs://criteo-datasets/criteo_processed_parquet/workflow/categories/unique.C16.parquet...
Copying gs://criteo-datasets/criteo_processed_parquet/workflow/categories/unique.C18.parquet...
Copying gs://criteo-datasets/criteo_processed_parquet/workflow/categories/unique.C17.parquet...
Copying gs://criteo-datasets/criteo_proce

In [9]:
ensemble_config = EnsembleConfig()
local_workflow_path = Path(LOCAL_WORKSPACE) / Path(WORKFLOW_MODEL_DIR).parts[-1]
local_saved_model_path = Path(LOCAL_WORKSPACE) / Path(HUGECTR_MODEL_DIR).parts[-1]
local_ensemble_path = Path(LOCAL_WORKSPACE) / f'triton-ensemble-{time.strftime("%Y%m%d%H%M%S")}'
#model_registry_path = '/models'
model_repository_path = 'gs://jk-merlin-dev/ensembles/ensemble_1'


export.export_ensemble(
    model_name=MODEL_NAME,
    workflow_path=local_workflow_path,
    saved_model_path=local_saved_model_path,
    output_path=local_ensemble_path,
    categorical_columns=ensemble_config.categorical_columns,
    continuous_columns=ensemble_config.continuous_columns,
    label_columns=ensemble_config.label_columns,
    num_slots=ensemble_config.num_slots,
    max_nnz=ensemble_config.max_nnz,
    num_outputs=ensemble_config.num_outputs,
    embedding_vector_size=ensemble_config.embedding_vector_size,
    max_batch_size=ensemble_config.max_batch_size,
    model_registry_path=model_repository_path
    )